In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from keys import API_KEY

In [32]:
def get_energy_data_batches():
    url = "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/"

    period_test = []

    group_1 = [1,3,5,7,8,10,12]
    group_2 = [4,6,9,11]

    for i in range(1, 13):
        month_day_decider = 0
        if i in group_1:
            month_day_decider = 32
        elif i in group_2:
            month_day_decider = 31
        else:
            month_day_decider = 29
        if i < 10:
            placeholder_str = f"0{i}"
        else:
            placeholder_str = f"{i}"
        for x in range(1, month_day_decider):
            if x < 10:
                period_test.append((f"2024-{placeholder_str}-0{x}T00", f"2024-{placeholder_str}-0{x}T23"))
            else:
                period_test.append((f"2024-{placeholder_str}-{x}T00", f"2024-{placeholder_str}-{x}T23"))
    print(len(period_test))


    time_periods = [
        ("2025-05-31T00", "2025-05-31T23"),
        ("2025-06-1T00", "2025-06-1T23")
    ]
    
    all_data = []
    total_records = 0
    
    print("Starting data collection process...")
    
    for start_date, end_date in period_test:
            
        params = {
            "api_key": API_KEY,
            "frequency": "hourly",
            "data[0]": "value",
            "start": start_date,
            "end": end_date,
            "sort[0][column]": "period",
            "sort[0][direction]": "asc",
            "length": 5000
        }
        
        print(f"Requesting data from {start_date} to {end_date}...")
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if 'response' in data and 'data' in data['response']:
                batch = pd.DataFrame(data['response']['data'])
                batch_size = len(batch)
                
                if batch_size > 0:
                    all_data.append(batch)
                    total_records += batch_size
                    print(f"Retrieved {batch_size} records. Total collected: {total_records}")
                else:
                    print(f"No data returned for period {start_date} to {end_date}")
        else:
            print(f"API request failed with status code {response.status_code}")
        
    
    if not all_data:
        print("No data was collected.")
        return None
        
    energy_df = pd.concat(all_data, ignore_index=True)
    print(f"Successfully collected a total of {len(energy_df)} records")

    energy_df['period'] = pd.to_datetime(energy_df['period'])
    energy_df['value'] = pd.to_numeric(energy_df['value'], errors='coerce')
    
    return energy_df

energy_df = get_energy_data_batches()

if energy_df is not None and not energy_df.empty:
    print("\nData collection successful!")
    print(f"Dataset shape: {energy_df.shape}")

else:
    print("Data collection failed.")

365
Starting data collection process...
Requesting data from 2024-01-01T00 to 2024-01-01T23...
Retrieved 390 records. Total collected: 390
Requesting data from 2024-01-02T00 to 2024-01-02T23...
Retrieved 390 records. Total collected: 780
Requesting data from 2024-01-03T00 to 2024-01-03T23...
Retrieved 390 records. Total collected: 1170
Requesting data from 2024-01-04T00 to 2024-01-04T23...
Retrieved 390 records. Total collected: 1560
Requesting data from 2024-01-05T00 to 2024-01-05T23...
Retrieved 390 records. Total collected: 1950
Requesting data from 2024-01-06T00 to 2024-01-06T23...
Retrieved 392 records. Total collected: 2342
Requesting data from 2024-01-07T00 to 2024-01-07T23...
Retrieved 390 records. Total collected: 2732
Requesting data from 2024-01-08T00 to 2024-01-08T23...
Retrieved 390 records. Total collected: 3122
Requesting data from 2024-01-09T00 to 2024-01-09T23...
Retrieved 390 records. Total collected: 3512
Requesting data from 2024-01-10T00 to 2024-01-10T23...
Retriev

In [ ]:
def get_energy_data_batches():
    url = "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/"

    # Example: fetch all of 2023
    start_date = "2023-01-01T00"
    end_date = "2023-12-31T23"

    len_of_request = 5000
    offset = 0
    total_records = 0

    params = {
        "api_key": API_KEY,
        "frequency": "hourly",
        "data[0]": "value",
        "start": start_date,
        "end": end_date,
        "sort[0][column]": "period",
        "sort[0][direction]": "asc",
        "length": len_of_request
    }

    all_data = []

    print(f"Requesting monthly data from {start_date} to {end_date}...")
    print("Starting paginated data collection....")
    while True:
        params_copy = params.copy()
        params_copy['offset'] = offset
        print(f"Requesting {offset} to {len_of_request + offset - 1} rows...")
        response = requests.get(url, params=params_copy)
        if response.status_code != 200:
            print("Could not fetch the data from the source")
        
        data = response.json()
        if 'response' in data and 'data' in data['response']:
            batch = pd.DataFrame(data['response']['data'])
            print(f"Retrieved {len(batch)} records.")
            if batch.empty:
                print("No data returned")
                break
            all_data.append(batch)
            batch_size = len(batch)
            total_records += batch_size
            print(f"Retrieved {batch_size} data and data amount totalled to {total_records}")
            if batch_size < len_of_request:
                print(f"Last batch is collected. Ending pagination.")
                break
            offset += batch_size
        else:
            print("No data returned for this batch.")
            break
    if batch.empty:
        print("No data was collected")
        return None

    energy_df = pd.concat(all_data, ignore_index=True)
    print(f"sucessfully collected {len(energy_df)} records")
    energy_df['period'] = pd.to_datetime(energy_df['period'])
    energy_df['value'] = pd.to_numeric(energy_df['value'], errors='coerce')

    return energy_df

energy_df = get_energy_data_batches()

if energy_df is not None and not energy_df.empty:
    print("\nData collection successful!")
    print(f"Dataset shape: {energy_df.shape}")
else:
    print("Data collection failed.")

Requesting monthly data from 2023-01-01T00 to 2023-12-31T23...
Starting paginated data collection....
Requesting 0 to 4999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 5000
Requesting 5000 to 9999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 10000
Requesting 10000 to 14999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 15000
Requesting 15000 to 19999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 20000
Requesting 20000 to 24999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 25000
Requesting 25000 to 29999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 30000
Requesting 30000 to 34999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 35000
Requesting 35000 to 39999 rows...
Retrieved 5000 records.
Retrieved 5000 data and data amount totalled to 40000
Requesting

In [53]:
energy_df

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2023-01-01,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,1956.0,megawatthours
1,2023-01-01,AECI,"Associated Electric Cooperative, Inc.",NG,Natural Gas,971.0,megawatthours
2,2023-01-01,AECI,"Associated Electric Cooperative, Inc.",WND,Wind,155.0,megawatthours
3,2023-01-01,AVA,Avista Corporation,NG,Natural Gas,214.0,megawatthours
4,2023-01-01,AVA,Avista Corporation,OTH,Other,111.0,megawatthours
...,...,...,...,...,...,...,...
9687,2023-01-02,WALC,Western Area Power Administration - Desert Sou...,WAT,Hydro,192.0,megawatthours
9688,2023-01-02,WALC,Western Area Power Administration - Desert Sou...,WND,Wind,97.0,megawatthours
9689,2023-01-02,WAUW,Western Area Power Administration - Upper Grea...,WAT,Hydro,69.0,megawatthours
9690,2023-01-02,WWA,"NaturEner Wind Watch, LLC",WND,Wind,46.0,megawatthours


In [25]:
if energy_df is not None and not energy_df.empty:
    print("Hour distribution in energy_df AFTER dropna():")
    print(energy_df['period'].dt.hour.value_counts().sort_index())
else:
    print("energy_df is empty or None after dropna().")

Hour distribution in energy_df AFTER dropna():
period
0    142866
Name: count, dtype: int64


In [35]:
energy_df.tail()

,period,respondent,respondent-name,fueltype,type-name,value,value-units,hour
142861,2024-12-31,WALC,Western Area Power Administration - Desert Sou...,WAT,Hydro,0.0,megawatthours,0
142862,2024-12-31,WALC,Western Area Power Administration - Desert Sou...,WND,Wind,81.0,megawatthours,0
142863,2024-12-31,WAUW,Western Area Power Administration - Upper Grea...,WAT,Hydro,69.0,megawatthours,0
142864,2024-12-31,WWA,"NaturEner Wind Watch, LLC",WND,Wind,0.0,megawatthours,0
142865,2024-12-31,YAD,"Alcoa Power Generating, Inc. - Yadkin Division",WAT,Hydro,123.0,megawatthours,0


In [26]:
val_mean = energy_df["value"].mean()
energy_df['value'] = energy_df['value'].fillna(val_mean)

In [27]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142866 entries, 0 to 142865
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   period           142866 non-null  datetime64[ns]
 1   respondent       142866 non-null  object        
 2   respondent-name  142866 non-null  object        
 3   fueltype         142866 non-null  object        
 4   type-name        142866 non-null  object        
 5   value            142866 non-null  float64       
 6   value-units      142866 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 7.6+ MB


In [28]:
total_electric_used = energy_df['value'].sum()
total_electric_used

577939053.5139146

In [32]:
energy_df["hour"] = energy_df["period"].dt.hour

In [34]:
print("Distribution of hours in the original data:")
print(energy_df['period'].dt.hour.value_counts().sort_index())

# Check a few sample rows to see the actual timestamps
print("\nSample rows with period values:")
sample_rows = energy_df.sample(min(10, len(energy_df)))
print(sample_rows[['period', 'fueltype', 'value']])

# Check if all periods are on the same date(s)
print("\nUnique dates in the dataset:")
print(energy_df['period'].dt.date.value_counts().head())

# Examine how API calls were made
print("\nChecking your API call parameters:")
print("First few period_test tuples:")
for i, (start, end) in enumerate(period_test[:5]):
    print(f"{i}: {start} to {end}")

Distribution of hours in the original data:
period
0    142866
Name: count, dtype: int64

Sample rows with period values:
           period fueltype    value
91177  2024-08-23      WND    315.0
80637  2024-07-27      OTH    601.0
132537 2024-12-06      SUN     36.0
26239  2024-03-09       NG  27963.0
99196  2024-09-13       NG   1007.0
41203  2024-04-16      SUN     15.0
120564 2024-11-07      BAT   1865.0
82119  2024-07-31      OTH    671.0
128587 2024-11-26      WAT  33067.0
44     2024-01-01      WAT   1172.0

Unique dates in the dataset:
period
2024-12-31    413
2024-12-30    412
2024-12-27    409
2024-12-26    409
2024-12-25    409
Name: count, dtype: int64

Checking your API call parameters:
First few period_test tuples:


NameError: name 'period_test' is not defined

In [33]:
data = energy_df.groupby(["hour", "fueltype"])["value"].sum()
data_table = data.unstack(fill_value=0)
data_table.head(100)
print(data_table.index.unique()) 

Index([0], dtype='int32', name='hour')
